In [ ]:
from google.colab.patches import cv2_imshow
import cv2 as cv
from google.colab import drive
import numpy as np
import os

# Connect to Google Drive
drive.mount('/content/drive/')

# Path to the model
model_path = "/content/drive/MyDrive/graph_opt.pb"

# Load the model
net = cv.dnn.readNetFromTensorflow(model_path)

# Define input width and height
inWidth = 368
inHeight = 368

# Threshold for confidence
thr = 0.2

# Define body parts and pairs
BODY_PARTS = { "Nose": 0, "Neck": 1, "RShoulder": 2, "RElbow": 3, "RWrist": 4,
               "LShoulder": 5, "LElbow": 6, "LWrist": 7, "RHip": 8, "RKnee": 9,
               "RAnkle": 10, "LHip": 11, "LKnee": 12, "LAnkle": 13, "REye": 14,
               "LEye": 15, "REar": 16, "LEar": 17, "Background": 18 }

POSE_PAIRS = [ ["Neck", "RShoulder"], ["Neck", "LShoulder"], ["RShoulder", "RElbow"],
               ["RElbow", "RWrist"], ["LShoulder", "LElbow"], ["LElbow", "LWrist"],
               ["Neck", "RHip"], ["RHip", "RKnee"], ["RKnee", "RAnkle"], ["Neck", "LHip"],
               ["LHip", "LKnee"], ["LKnee", "LAnkle"], ["Neck", "Nose"], ["Nose", "REye"],
               ["REye", "REar"], ["Nose", "LEye"], ["LEye", "LEar"] ]

# Path to the folder containing images
images_folder_path = "/content/drive/MyDrive/skeleton-pose/silhouette-pd"

# Get list of image filenames
image_files = os.listdir(images_folder_path)

# Sort image files
image_files.sort()

# Main loop to process images
for image_file in image_files:
    # Read image
    frame = cv.imread(os.path.join(images_folder_path, image_file))

    # Get frame dimensions
    frameWidth = frame.shape[1]
    frameHeight = frame.shape[0]

    # Set input to the network
    net.setInput(cv.dnn.blobFromImage(frame, 1.0, (inWidth, inHeight), (127.5, 127.5, 127.5), swapRB=True, crop=False))
    out = net.forward()
    out = out[:, :19, :, :]  # MobileNet output [1, 57, -1, -1], we only need the first 19 elements

    assert(len(BODY_PARTS) == out.shape[1])

    points = []
    for i in range(len(BODY_PARTS)):
        # Slice heatmap of corresponding body part.
        heatMap = out[0, i, :, :]

        # Find local maximums
        _, conf, _, point = cv.minMaxLoc(heatMap)
        x = (frameWidth * point[0]) / out.shape[3]
        y = (frameHeight * point[1]) / out.shape[2]
        # Add a point if its confidence is higher than threshold.
        points.append((int(x), int(y)) if conf > thr else None)

    for pair in POSE_PAIRS:
        partFrom = pair[0]
        partTo = pair[1]
        assert(partFrom in BODY_PARTS)
        assert(partTo in BODY_PARTS)

        idFrom = BODY_PARTS[partFrom]
        idTo = BODY_PARTS[partTo]

        if points[idFrom] and points[idTo]:
            cv.line(frame, points[idFrom], points[idTo], (0, 255, 0), 3)
            cv.ellipse(frame, points[idFrom], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)
            cv.ellipse(frame, points[idTo], (3, 3), 0, 0, 360, (0, 0, 255), cv.FILLED)

    cv2_imshow(frame)

    # Write the frame to the folder
    cv.imwrite(os.path.join(images_folder_path, f"pose_{image_file}"), frame)
